In [1]:
import pandas as pd 
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
#from fbprophet import Prophet
import datetime 
import warnings 
from fbprophet import Prophet
warnings.simplefilter('ignore')

ModuleNotFoundError: No module named 'seaborn'

In [2]:
df = pd.read_csv("../dataset/outputs.csv")

In [3]:
def change_date_format(date):
    """We want to have this format for the prophet API : YYYY-MM-DD HH:MM:SS"""
    return datetime.datetime.strptime(date, '%d/%m/%Y %H:%M').strftime('%Y-%m-%d %H:%M:00')

In [4]:
mydf=df[['Horodate_UTC','RES1_BASE']]
mydf["Horodate_UTC"]=mydf["Horodate_UTC"].apply(change_date_format)
mydf=mydf.rename(columns={"Horodate_UTC":"ds","RES1_BASE":"y"})

In [6]:
df_train,df_test = mydf[:65130],df[65130:]

In [ ]:
m=Prophet(interval_width=0.95,changepoint_prior_scale=0.01).fit(mydf)